In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
from pathlib import Path

In [ ]:
sns.set_style("darkgrid")

In [ ]:
def read_results(path):
    json_path = Path(path)
    data = json.loads(json_path.read_text())

    # Remove algorithm params for simplicity
    for x in data:
        x['job']['algo_name'] = list(x['job']['algo'].keys())[0]
        x['job']['algo_full'] = str(x['job']['algo'])
        del x['job']['algo']
    df = pd.json_normalize(data, sep='_')

    return df

In [ ]:
df = read_results("../results/affine.json")

In [ ]:
df = df[(df.job_costs_sub == 1) & (df.job_costs_open == 1) & (df.job_costs_extend == 1)]
df = df[df.output_measured_runtime.notna()]
df = df[df.job_meta_error_model == 'Uniform']

In [ ]:
df_trace = df[df.job_traceback == True]
plot = sns.scatterplot(
    data=df_trace,
    x='job_meta_length',
    y='output_measured_runtime',
    hue='job_algo_full',
    style='job_algo_full',
    size='job_meta_error_rate',
    legend='full',
    alpha=0.5
)
sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))
plot.set(xscale='log', yscale='log')
plt.title("score and traceback")
plt.savefig('plot_trace.png', bbox_inches='tight')
plt.show()

In [ ]:
df_score = df[df.job_traceback == False]
plot = sns.scatterplot(
    data=df_score,
    x='job_meta_length',
    y='output_measured_runtime',
    hue='job_algo_full',
    style='job_algo_full',
    size='job_meta_error_rate',
    legend='full',
    alpha=0.5
)
sns.move_legend(plot, "upper left", bbox_to_anchor=(1, 1))
plot.set(xscale='log', yscale='log')
plt.title("score only")
plt.savefig('plot_score.png', bbox_inches='tight')
plt.show()